# IFT6390 Project - Data Exploration snippets

## Useful piece of code

## Setup - data

In [2]:
import pandas as pd
import numpy as np

## Import cleaned dataframe

In [4]:
s140=pd.read_pickle('data/s140_clean_28nov.pkl')
mr=pd.read_pickle('data/mr_clean_28nov.pkl')
cc=pd.read_pickle('data/cc_clean_28nov.pkl')

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer 

def make_tfidf(df):
    tf_vect=TfidfVectorizer(use_idf=False,stop_words=stopw)
    tf=tf_vect.fit_transform(df.tolist())
    
    l=tf.shape[0]
    
    w_count=np.array(tf.sum(axis=0,))[0]/l
    #wcm=w_count.max()
    #w_count=w_count/wcm
    wcr=w_count.argsort()[::-1].argsort()

    
    idf_vect=TfidfVectorizer(use_idf=True,stop_words=stopw)
    idf=idf_vect.fit_transform(df.tolist())
    
    w_marker=np.array(idf.sum(axis=0))[0]/l
    #wmm=w_marker.max()
    #w_marker=w_marker/wmm
    wmr=w_marker.argsort()[::-1].argsort()
    
    
    # SRC -> https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/
    feature_names=tf_vect.get_feature_names()
    
    tf_idf = pd.DataFrame(np.array([w_count, w_marker, wcr, wmr]).T, index=feature_names, columns=["tf", "idf", "rank_tf", "rank_idf"])
    tf_idf = tf_idf.sort_values(by=["tf"],ascending=False)

    tf_idf.rank_tf=tf_idf.rank_tf.astype('int64')
    tf_idf.rank_idf=tf_idf.rank_idf.astype('int64')

    return tf_idf


In [8]:
all=make_tfidf(s140['trimmed'])

In [9]:
tf_idf_vect=TfidfVectorizer(use_idf=True,stop_words=stopw)
tf_idf=tf_idf_vect.fit_transform(s140['trimmed'].tolist())

## Train s140

In [12]:
import sys
!{sys.executable} -m pip install scikit-plot

import scikitplot#.plotters as skplt

## Model testing

In [10]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

#The mean score and the 95% confidence interval of the score estimate are hence given by:

def TryModel(model, x, y):    
    if True:
        X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.10, random_state=42)
        
        clf = model.fit(X_train, y_train)
        
        predicted = clf.predict(X_valid)

        from sklearn.metrics import accuracy_score
        acc=accuracy_score(y_valid, predicted)
        print(f"Accuracy: {acc}")
    else:
        scores = cross_val_score(model, x, y, cv=5)
        print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

    scikitplot.metrics.plot_confusion_matrix(y_valid, predicted,x_tick_rotation=90,figsize=(9,9))#, normalize=True)
    
    plt.show()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

x,y = tf_idf, s140['target']

#TryModel(MultinomialNB(), x, y)
#TryModel(linear_model.SGDClassifier(max_iter=1000, tol=1e-3), x, y)
TryModel(MLPClassifier(alpha=0.25, max_iter=25,epsilon=1e-02,verbose=True), x, y)
#TryModel(AdaBoostClassifier(DecisionTreeClassifier(max_depth=10)), x, y)
#vch = VotingClassifier(estimators=[('NB25', clf1), ('SGDp', clf2), ('SGDlog', clf3),('NB35', clf4)], voting='hard')
#vcs = VotingClassifier(estimators=[('NB10', clf1), ('SGD', clf2), ('SGDlog', clf3),('NB35', clf4)], voting='soft')
#TryModel(svm.SVC(kernel='linear', C=1), x, y)
